In [5]:
!pip install youtube-search-python

In [7]:
!pip install --upgrade youtube-search-python youtube-transcript-api transformers torch httpx==0.24.0

import torch
from youtubesearchpython import VideosSearch
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline

def search_youtube(query, max_results=5):
    search = VideosSearch(query, limit=max_results)
    results = search.result()['result']
    videos = []
    for r in results:
        videos.append({
            "title": r["title"],
            "id": r["id"],
            "url": r["link"]
        })
    return videos


def get_transcript(video_id, lang="en"):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])
        text = " ".join([t["text"] for t in transcript])
        return text
    except Exception as e:
        return f"Transcript not available: {e}"

def summarize_text(text, model_name="facebook/bart-large-cnn"):
    summarizer = pipeline("summarization", model=model_name, device=0 if torch.cuda.is_available() else -1)


    max_chunk = 900
    sentences = text.split(". ")
    chunks, current_chunk = [], ""
    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) <= max_chunk:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk)
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk)


    summary = []
    for chunk in chunks:
        summary_text = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summary.append(summary_text)

    return " ".join(summary)

query = input(" Enter a topic to search on YouTube: ")
videos = search_youtube(query)

print("\n Top results:")
for i, v in enumerate(videos):
    print(f"{i+1}. {v['title']} - {v['url']}")

choice = int(input("\n Enter the number of the video you want to summarize: ")) - 1
video_id = videos[choice]["id"]

transcript_text = get_transcript(video_id)
if "Transcript not available" in transcript_text:
    print("\n", transcript_text)
else:
    print("\n Summarizing the video... please wait...")
    summary = summarize_text(transcript_text)
    print("\n Summary:\n", summary)

print("\nIf you don't get a summary for your video, there may not be a transcript available for it.")

 Enter a topic to search on YouTube: Neil deGrasse

 Top results:
1. Almost 3 Hours of THE BEST Mind Blowing Facts with Neil deGrasse Tyson! - https://www.youtube.com/watch?v=grb7cR3QHTQ
2. ULTIMATE 90 Minutes Of Mind Blowing Facts! From Dr. Neil deGrasse Tyson - https://www.youtube.com/watch?v=OwAk01PbCYI
3. Why We’re Putting Nuclear Reactors on the Moon - https://www.youtube.com/watch?v=RWL2efggMY0
4. The Intersection of Science and Religion with Neil deGrasse Tyson - https://www.youtube.com/watch?v=v_TLvPmzuN0
5. Neil deGrasse Tyson: Do THIS Every Morning To Find Happiness & Meaning In Your Life! - https://www.youtube.com/watch?v=_yZiNnQftxU

 Enter the number of the video you want to summarize: 1

 Transcript not available: type object 'YouTubeTranscriptApi' has no attribute 'get_transcript'

If you don't get a summary for your video, there may not be a transcript available for it.
